In [ ]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()
alt.renderers.set_embed_options(padding={"left": 0, "right": 0, "bottom": 0, "top": 0})

In [ ]:
calls = pd.read_csv(snakemake.input[0], sep="\t")
# calls = pd.read_csv("/local/snakemake-workflow-sars-cov2/results/benchmarking/tables/aggregated_read_calls.tsv", sep="\t")
ordered_state = {
    "read": "1. Kallisto/Reads",
    "contig": "2. Pangoling/Contig",
    "scaffold": "3. Pangolin/Scaffold",
    "polished_scaffold": "4. Pangolin/Polished Scaffold",
    "pseudo": "5. Pangolin/Pseudo",
}
calls["state"].replace(ordered_state, inplace=True)

In [ ]:
sort_order = ["read", "contig", "scaffold", "polished_scaffold", "pseudo"]

heatmap = (
    alt.Chart(calls)
    .mark_bar()
    .encode(
        y=alt.Y(
            "read_number:O", axis=alt.Axis(format=",.0f"), title="Read Count per Sample"
        ),
        x=alt.X("read_length:O", title="Read Length"),
        color=alt.Color(
            "sum(correct_call):Q",
            title="Lineage correctly called [%]",
            legend=alt.Legend(orient="bottom"),
        ),
        tooltip=["sum(correct_call)", "count(correct_call)"],
        text="sum(correct_call):Q",
    )
)

text = (
    alt.Chart()
    .mark_text(baseline="middle", fontSize=7)
    .encode(
        y="read_number:O",
        x="read_length:O",
        text="sum(correct_call):Q",
        color=alt.value("white"),
    )
)

# Draw the chart.
alt.layer(heatmap, text, data=calls).facet(
    facet=alt.Facet(
        "state",
        title="Lineage called on",
    ),
).configure_scale(bandPaddingInner=0.01).save(snakemake.output[0])